<a href="https://colab.research.google.com/github/arsey99999/-_-/blob/main/%EA%B0%95%EC%82%AC%EB%8B%98_dacon_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from google.colab import files
uploded = files.upload()

train = pd.read_csv('train.csv')

Saving train.csv to train.csv


In [ ]:
train = pd.read_csv('./train.csv')

In [ ]:
train.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일,허위매물여부
0,TRAIN_0000,현장확인,402500000.0,470000,NaN,NaN,15.0,서향,1.0,1.0,가능,40.0,96,t93Nt6I2I0,B플랫폼,2024-10-09,0
1,TRAIN_0001,현장확인,170500000.0,200000,NaN,3.0,4.0,남동향,2.0,1.0,불가능,NaN,0,q39iV5J4E6,D플랫폼,2024-12-26,0
2,TRAIN_0002,전화확인,114000000.0,380000,NaN,2.0,3.0,동향,1.0,1.0,불가능,NaN,0,b03oE4G3F6,A플랫폼,2024-11-28,0
3,TRAIN_0003,현장확인,163500000.0,30000,36.3,3.0,9.0,남동향,2.0,1.0,가능,13.0,10,G52Iz8V2B9,A플랫폼,2024-11-26,0
4,TRAIN_0004,현장확인,346000000.0,530000,NaN,3.0,3.0,동향,2.0,1.0,불가능,NaN,0,N45gM0M7R0,B플랫폼,2024-06-25,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2452 non-null   object 
 1   매물확인방식  2452 non-null   object 
 2   보증금     2452 non-null   float64
 3   월세      2452 non-null   int64  
 4   전용면적    1665 non-null   float64
 5   해당층     2223 non-null   float64
 6   총층      2436 non-null   float64
 7   방향      2452 non-null   object 
 8   방수      2436 non-null   float64
 9   욕실수     2434 non-null   float64
 10  주차가능여부  2452 non-null   object 
 11  총주차대수   1756 non-null   float64
 12  관리비     2452 non-null   int64  
 13  중개사무소   2452 non-null   object 
 14  제공플랫폼   2452 non-null   object 
 15  게재일     2452 non-null   object 
 16  허위매물여부  2452 non-null   int64  
dtypes: float64(7), int64(3), object(7)
memory usage: 325.8+ KB


In [ ]:
x = train.drop(['ID','허위매물여부'], axis=1)
y = train['허위매물여부']

In [ ]:
# SimpleImputer : 평균 대체
mean_imputer = SimpleImputer(strategy='mean')

# 결측값을 평균으로 대체할 열 목록
columns_fill_mean = ['해당층', '총층','전용면적','방수', '욕실수','총주차대수']

# 학습 데이터에 fit 및 transform
x[columns_fill_mean] = mean_imputer.fit_transform(x[columns_fill_mean])

In [ ]:
# Label Encoding 적용 열
label_encode_cols = ['중개사무소','게재일','제공플랫폼','방향']

label_encoders = {} # ??????? 언젠가 다시 돌아온다
for col in label_encode_cols:
    le = LabelEncoder()
    x[col] = le.fit_transform(x[col].astype(str))
    label_encoders[col] = le

In [ ]:
# A, B, C
# A: => 1 0 0
# B: 0 1 0
# C: 0 0 1
# sparse <> dense

In [ ]:
# One-Hot Encoding 적용 열
one_hot_cols = ['매물확인방식', '주차가능여부']

# One-Hot Encoding 적용
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [ ]:
# Train 데이터 변환
x_encoded = one_hot_encoder.fit_transform(x[one_hot_cols])
x_encoded

array([[0., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 0., 1.],
       ...,
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1.]])

In [ ]:
one_hot_encoder.get_feature_names_out(one_hot_cols)

array(['매물확인방식_서류확인', '매물확인방식_전화확인', '매물확인방식_현장확인', '주차가능여부_가능',
       '주차가능여부_불가능'], dtype=object)

In [ ]:
x_encoded_df = pd.DataFrame(x_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=x.index)

In [ ]:
# 기존 데이터와 병합
x = pd.concat([x.drop(columns=one_hot_cols), x_encoded_df], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

# train test split
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2,
                                                  random_state=999,
                                                  stratify=y)


In [ ]:
model = RandomForestClassifier(n_estimators=100,
                               criterion='gini',
                               max_depth=None,
                               random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)

f1_macro = f1_score(y_val, y_pred, average="macro")

print("accuracy : {:.2f}%".format(accuracy*100))
print("f1 score : {:.2f}%".format(f1_macro*100))

accuracy : 96.13%
f1 score : 90.20%


In [ ]:
# TODO:
# 하이퍼파라미터튜닝 하고 다시 validation f1_score 구하기
# How?
# https://hul980.tistory.com/16
# GridSearchCV 써서 최고의 파라미터 구해라

In [ ]:
params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv=GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

In [ ]:
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.9439


In [ ]:
model = RandomForestClassifier(n_estimators=100,
                               criterion='gini',
                               max_depth=20,
                               min_samples_leaf=8,
                               min_samples_split=8,
                               random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, min_samples_leaf=8, min_samples_split=8,
                       random_state=42)

In [ ]:
y_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)

f1_macro = f1_score(y_val, y_pred, average="macro")

print("accuracy : {:.2f}%".format(accuracy*100))
print("f1 score : {:.2f}%".format(f1_macro*100))

accuracy : 95.93%
f1 score : 89.42%


In [ ]:
from google.colab import files
import pandas as pd
from sklearn.impute import SimpleImputer

# 파일 업로드
uploaded = files.upload()

# test.csv 읽기
test = pd.read_csv('test.csv')

# 결측값 확인
missing_columns = test.columns[test.isnull().any()].tolist()
print(f"결측값이 있는 컬럼: {missing_columns}")

# 결측값 처리
if missing_columns:
    mean_imputer = SimpleImputer(strategy='mean')
    mean_imputer.fit(test[missing_columns])  # 결측값 대체를 위한 학습
    test[missing_columns] = mean_imputer.transform(test[missing_columns])

Saving test.csv to test (4).csv
결측값이 있는 컬럼: ['전용면적', '해당층', '총층', '방수', '욕실수', '총주차대수']


In [ ]:
test.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일
0,TEST_000,서류확인,223000000.0,530000,27.7638,5.0,5.0,남향,1.0,1.0,불가능,20.292237,7,Z86Th6S3K5,D플랫폼,2024-10-21
1,TEST_001,서류확인,150500000.0,590000,30.2600,7.0,11.0,서향,1.0,1.0,불가능,16.000000,11,G52Iz8V2B9,D플랫폼,2023-09-02
2,TEST_002,현장확인,47000000.0,200000,41.5000,2.0,3.0,남서향,2.0,1.0,불가능,20.292237,0,N45gM0M7R0,B플랫폼,2024-04-03
3,TEST_003,서류확인,133000000.0,250000,31.3500,5.0,6.0,남향,2.0,1.0,가능,20.292237,5,C41wx1K6U9,B플랫폼,2024-09-19
4,TEST_004,현장확인,108000000.0,380000,23.1400,2.0,3.0,남향,1.0,1.0,불가능,20.292237,0,Z68ZJ6F6L4,A플랫폼,2024-11-10


In [ ]:
# Label Encoding
for col in label_encode_cols:
    if col in test.columns:
        le = label_encoders[col]
        test[col] = test[col].astype(str)
        unseen = set(test[col].unique()) - set(le.classes_)
        # unseen = []

        if unseen: # 뜬금포가 있다
            le.classes_ = np.append(le.classes_, list(unseen))
        test[col] = le.transform(test[col].astype(str))

In [ ]:
# One-Hot Encoding
test_encoded = one_hot_encoder.transform(test[one_hot_cols])
test_encoded_df = pd.DataFrame(test_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=test.index)

test = pd.concat([test.drop(columns=one_hot_cols), test_encoded_df], axis=1)

In [ ]:
test.drop(columns=['ID'],inplace=True)

In [ ]:
test.tail()

NameError: name 'test' is not defined

In [ ]:
pred = pd.Series(model.predict(test))

In [ ]:
pred

,0
0,0
1,0
2,1
3,0
4,0
...,...
608,0
609,0
610,0
611,0


## 제출하기!

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission (1).csv


In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['허위매물여부'] = pred # 우리의 예측 넣는다
submit.head()

,ID,허위매물여부
0,TEST_000,0
1,TEST_001,0
2,TEST_002,1
3,TEST_003,0
4,TEST_004,0


In [ ]:
submit.to_csv('./baseline_submission.csv',index=False)